# Extração da característica de relação sinal-ruído de dados de EEG

A ideia é utilizar dados fictícios de ruído e sinal "bom"., para criarmos a relação dos dois sinais e obter como resultado um sinal de interesse "limpo".

A partir deste sinal, podemos no contexto de caracterização de foco, ainda extrair os rítmos cerebrais ou então classificar sinais com a presença ou não de foco, de forma que as amostras de sinais extraídas de um buffer sejam rotuladas com com a presença ou não de foco.

Esta atividade pode ser realizada em conjunto com um classificador comumente utilizado, como é o caso do SVM. Neste caso, uma porcetagem das amostras são utilizadas para treino e o restante para teste (p.e. 30 e 70% respectivamente).

In [1]:
# Criando dados fictícios
# pensando em um vídeo de 10 minutos, com uma taxa de amostragem de 250 Hz
#   teremos 10 * 60 * 250 = 150.000 amostras (sinal com foco)
#   teremos 1 * 60 * 250 = 15.000 amostras (basal)

import numpy as np


shape = (150000, 8)
data_focus = np.random.normal(loc=0, scale=10, size=shape).astype(np.float32)

shape = (15000, 8)
data_base = np.random.normal(loc=0, scale=10, size=shape).astype(np.float32)

In [2]:
# Estimando o ruído de fundo (utilizando o sinal basal)

# armazena uma lista com as médias de potência para cada canal
noise_power = []
for channel_data in data_base.T:
    fft_result = np.fft.fft(channel_data)
    # densidade espectral de potência (PSD)
    psd = np.abs(fft_result) ** 2
    # média da potência no intervalo de tempo sem estímulo
    base_power = np.mean(psd)
    noise_power.append(base_power)
# média das médias de potência de todos os canais para estimar o ruído de fundo
estimated_background_noise = np.mean(noise_power)
print(estimated_background_noise)

1488566.1448603985


In [3]:
# agora vamos adaptar ambas características 
# aplicando para o nosso sinal de interesse

# forçando (estragando) valor de "estimated_background_noise" para não sobrar valores negativos
target_amplitudes_adjusted = data_focus - estimated_background_noise

# subtraindo o ruído de fundo das amplitudes
narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)
print(narrow_band_SNR)
print(narrow_band_SNR.shape)

total_power = np.sum(target_amplitudes_adjusted)
wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)
print(wide_band_SNR)
print(wide_band_SNR.shape)

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
(150000, 8)
[[-60.791798 -60.791794 -60.791775 ... -60.791798 -60.791855 -60.791862]
 [-60.79182  -60.791782 -60.79178  ... -60.791836 -60.79181  -60.791836]
 [-60.791798 -60.79181  -60.79178  ... -60.791813 -60.791813 -60.791832]
 ...
 [-60.79181  -60.791798 -60.791832 ... -60.791817 -60.791763 -60.79184 ]
 [-60.79182  -60.791832 -60.79181  ... -60.79181  -60.791847 -60.7918  ]
 [-60.79184  -60.79178  -60.791817 ... -60.791794 -60.79177  -60.791824]]
(150000, 8)


/tmp/ipykernel_23641/2021880081.py:8: RuntimeWarning: invalid value encountered in log10
  narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)
